In [ ]:
import pandas as pd
import numpy as np

# Load cleaned dataset
df = pd.read_csv("data/new_clean/cleaned_online_retail.csv")

# Ensure InvoiceDate is parsed correctly as datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Add Revenue column
df['Revenue'] = df['Quantity'] * df['Price']

# Define snapshot date (1 day after last transaction)
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
print(f"Snapshot date for recency calculation: {snapshot_date.date()}")

# Group by Customer ID and calculate R, F, M
rfm = df.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'Invoice': 'nunique',                                     # Frequency
    'Revenue': 'sum'                                          # Monetary
})

# Rename columns
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'Invoice': 'Frequency',
    'Revenue': 'Monetary'
}, inplace=True)

# Optional: remove customers with Monetary <= 0 (should be rare after cleaning)
rfm = rfm[rfm['Monetary'] > 0]

# Preview
print("RFM table preview:")
print(rfm.head(10))
rfm.to_csv("data/rfm/rfm_table.csv")
